In [ ]:
from user_posting_emulation import AWSDBConnector
from sqlalchemy import inspect, text
import pandas as pd

In [ ]:
def get_topic_data(topic_table, num_rows=2000):
    dbc = AWSDBConnector().create_db_connector()
    table_names = inspect(dbc).get_table_names()
    if topic_table not in table_names:
        raise ValueError(f'Given table `{topic_table}` not found in the database.')

    with dbc.connect() as conn:
        query = text(f'SELECT * FROM {topic_table} LIMIT {num_rows};')
        result = conn.execute(query)

    data = []
    for row in result:
        data.append(dict(row._mapping))
    
    return data

df_pin = pd.json_normalize(get_topic_data('pinterest_data', num_rows=11154))
# df_geo = pd.json_normalize(get_topic_data('geolocation_data', num_rows=11154))
# df_user = pd.json_normalize(get_topic_data('user_data', num_rows=11154))

In [ ]:
import multiprocessing
import pyspark
from pyspark.sql import DataFrame
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
pyspark.__version__

In [ ]:
cfg = (
    pyspark.SparkConf()
    # Setting the master to run locally and with the maximum amount of cpu coresfor multiprocessing.
    .setMaster(f"local[{multiprocessing.cpu_count()//2}]")
    # Setting application name
    .setAppName("S3ToSparkConnection")
    # Setting config value via string
    # .set("spark.eventLog.enabled", False)
    # Setting environment variables for executors to use
    # .setExecutorEnv(pairs=[("VAR3", "value3"), ("VAR4", "value4")])
    # Setting memory if this setting was not set previously
    .setIfMissing("spark.executor.memory", "2g")
    .setMaster('local[*]')
)

# Getting a single variable
print(cfg.get("spark.executor.memory"))
# Listing all of them in string readable format
print(cfg.toDebugString())

In [ ]:
# session = pyspark.sql.SparkSession.builder.config(conf=cfg).getOrCreate()
sc = pyspark.SparkContext(conf=cfg)

In [ ]:
import pyspark.pandas as psd

In [ ]:
pin_dfs = psd.DataFrame(df_pin)
# geo_dfs = psd.DataFrame(df_geo)
# user_dfs = psd.DataFrame(df_user)

In [ ]:
pin_dfs = pin_dfs.replace(['', 'N/A', 'n/a', 'none', 'None'], None)

## Clean Pin Spark DataFrame

Replace empty entries and entries with no relevant data in each column with `Nones` \
Perform the necessary transformations on the `follower_count` to ensure every entry is a number. \
Make sure the data type of this column is an `int`. \
Ensure that each column containing numeric data has a numeric data type \
Clean the data in the `save_location` column to include only the save location path \
Rename the `index` column to `ind`. \
Reorder the `DataFrame` columns to have the following column order: \
    `ind` \
    `unique_id` \
    `title` \
    `description` \
    `follower_count` \
    `poster_name` \
    `tag_list` \
    `is_image_or_video` \
    `image_src` \
    `save_location` \
    `category`

### Make sure unique ids are correct format

In [ ]:
invalid_uuid_index = pin_dfs[pin_dfs['unique_id'].str.len() != 36].index
pin_dfs = pin_dfs.drop(invalid_uuid_index.to_list())
print(f'{len(invalid_uuid_index)} rows dropped due to invalid UUID.')

### Cleanup and transform `follower_count`

In [ ]:
regex = r'[0-9]{1,}[kM]?'
invalid_follower_count_index = pin_dfs[~pin_dfs['follower_count'].str.match(regex)].index
pin_dfs = pin_dfs.drop(invalid_follower_count_index.to_list())
print(f'{len(invalid_follower_count_index)} rows dropped due to invalid follower count.')

In [ ]:
pin_dfs.select('follower_count')